In [542]:
# bibliotecas
import pandas as pd
import time
# funções 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Random Forest
legenda: treino, teste, erro, tempo<br><br>
n_estimators=11, max_depth=27 -> 0.9557926155884157 0.8334794802644814 3931.966016272742 8.193240880966187<br> <br>
n_estimators=100, max_depth=27 -> 0.9650129200912126 0.8408169218841756 3873.6263369273256 79.8025176525116<br>
# Linear Regression
legenda: treino, teste, erro<br><br>
fit_intercept=True, n_jobs=-1 -> 0.6463556388680558 0.5734454636073181 6866.667619648236<br>

### Célula usada para encontrar os melhores hiperparâmetros da Random Forest

```best_error = float('inf')
best_est = 0
best_depth = 0

for est in range(10, 101, 5):
    for depth in range(20, 31):
        model = RandomForestRegressor(random_state=1506, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_test = model.predict(features_test)
        error = mean_squared_error(target_test, predictions_test)**0.5
        print(f"Validação de REQM para n_estimators={est}, depth={depth} é {error}")
        if error < best_error:
            best_error = error
            best_est = est
            best_depth = depth

print(f"Melhor REQM de teste: {best_error} com n_estimators={best_est} e max_depth={best_depth}")
```

# Melhor modelo

In [543]:
# carregando dataset
car_data = pd.read_csv('datasets/car_data.csv')

In [544]:
# retirando colunas que não serão utilizadas
car_data = car_data.drop(['date_posted', 'days_listed', 'year_posted', 'car_age'] , axis=1)

In [545]:
# transformando a coluna condition em numérica colocando "pesos" para cada condição
condition_mapping = {
    "new": 5,
    "like new": 4,
    "excellent": 3,
    "good": 2,
    "fair": 1,
    "salvage": 0
}

# atribuindo os valores numéricos ao dataset
car_data['condition'] = car_data['condition'].map(condition_mapping)

In [546]:
# separando features e target
features = car_data.drop('price', axis=1)
target = car_data['price']

In [547]:
# aplicando One-Hot Encoding nas variáveis categóricas restantes
features_encoded = pd.get_dummies(features, columns=['model', 'fuel', 'transmission', 'type', 'paint_color', 'brand'])

In [548]:
# separando dados de treino e teste
features_train, features_test, target_train, target_test = train_test_split(features_encoded, target, test_size=0.2)

In [549]:
model = LinearRegression(fit_intercept=True, n_jobs=-1)

In [550]:
start = time.time()
model.fit(features_train, target_train)
end = time.time()
tempo = end - start
tempo

0.37096381187438965

In [551]:
print(model.score(features_train, target_train),
model.score(features_test, target_test))

0.6463556388680558 0.5734454636073181


In [552]:
predictions = model.predict(features_test)

In [553]:
rmse = mean_squared_error(target_test, predictions)** 0.5
rmse

6866.667619648236